In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


### just try 17 feature to predict next pm2.5,

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [4]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [5]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [6]:
year_data1=year_data.copy()

In [7]:
#### 台湾温度不会低于0度

In [8]:
# temperature always >=0 in Taiwan and change fast when temperature <=0 ,so correct it
for idx in [0]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <=5:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >5:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
---------- pm2.5 start : 
 -3.1 

-8.0
-7.2
-6.8
-6.5
-7.1
-7.4
-8.1
-8.3
-8.4
-9.3
-10.6
-11.2
-12.1
0.0
0.0
0.0
0.0
-12.3
pm2.5 end ------------
correct value between  : 19.0 18.0
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11


In [9]:
min(478,480)

478

In [10]:
year_data1[0].shape

(18, 480)

In [11]:
year_data1[3][9,year_data1[3][9]<0]

array([-1., -1., -1.])

In [12]:
for idx in [1, 2, 4, 6, 9, 12, 13]:
    for m in range(12):
        print(' month : ', m)
        i = 0
        while i < 480:
            if year_data1[m][idx, i] < 0:
                print('---------- pm2.5 start : \n', year_data1[m][idx, i],
                      '\n')
                for k in range(30):
                    if k + i + 1 > 479:
                        break
                    else:
                        if year_data1[m][idx, k + i + 1] >= 0:
                            break
                        else:
                            print(year_data1[m][idx, k + i + 1])
                i = i + k
                print('pm2.5 end ------------')
                print(idx,i-1-k,i+1)
                if year_data1[m][idx, min(i + 1,479)]==0:
                    year_data1[m][idx, min(i + 1,479)]=1
                if i+1>=480:
                    print('correct value between  :',year_data1[m][idx, i - 1 - k], year_data1[m][idx, i - 1 - k])
#                     year_data1[m][idx, 479]=year_data1[m][idx, i - 1 - k]
                    year_data1[m][idx,i - k:] = year_data1[m][idx, i - 1 - k]
                else:
                    print('correct value between  :',year_data1[m][idx, i - 1 - k], year_data1[m][idx, i + 1])
                ## add correct to mean value
                    year_data1[m][idx,i - k:i + 1] = (year_data1[m][idx, i - 1 - k] +year_data1[m][idx, i + 1]) / 2
            i = i + 1

 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.2 

-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
pm2.5 end ------------
1 73 83
correct value between  : 1.8 1.8
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.12 

pm2.5 end ------------
2 73 75
correct value between  : 0.34 0.26
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
4 26 28
correct value between  : 2.5 0.3
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
4 35 37
correct value between  : 0.1 1.0
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -1.1 

pm2.5 end ------------
4 74 76
correct value between  : 6.6 1.2
 month :  0
 month :  1
 mont

In [13]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [14]:
from scipy.stats import boxcox,boxcox_normmax
from scipy.special import boxcox1p

In [15]:
all_train_data2=pd.DataFrame()
for i in range(all_train_data1.shape[1]):
    if i != 9:
        lm=boxcox_normmax(all_train_data1[i]+1)
        all_train_data2[i]=boxcox1p(all_train_data1[i],lm)
    else:
        all_train_data2[i]=np.sqrt(all_train_data1[i])
        
    
    print(all_train_data1.index[i],'--skew : ',all_train_data1[i].skew(),'--kurt : ',all_train_data1[i].kurt())
    print('lambda : ',lm)
    print(all_train_data2.index[i],'--skew : ',all_train_data2[i].skew(),'--kurt : ',all_train_data2[i].kurt())
    print('--------------------------------------------------------------------')

0 --skew :  -0.2834134242907861 --kurt :  -0.7968009844193018
lambda :  1.461645538288768
0 --skew :  -0.07414275326700835 --kurt :  -0.9258821506244836
--------------------------------------------------------------------
1 --skew :  -5.095255676523535 --kurt :  85.70760213466164
lambda :  5.990940784090655
1 --skew :  0.2052435896115669 --kurt :  0.583625236412292
--------------------------------------------------------------------
2 --skew :  9.165303973318487 --kurt :  132.25692190213024
lambda :  -3.259330902857795
2 --skew :  -0.003907562716963148 --kurt :  -0.06972890461562153
--------------------------------------------------------------------
3 --skew :  2.172048918138678 --kurt :  8.458548044518952
lambda :  -6.182620739009405
3 --skew :  0.05859262861967747 --kurt :  -0.5568735662256521
--------------------------------------------------------------------
4 --skew :  5.378836580918865 --kurt :  41.25921290333162
lambda :  -0.7826645126628972
4 --skew :  -0.03484059806052529 --

In [16]:
# all_train_data3=pd.DataFrame()
# for i in range(all_train_data2.shape[1]):
#     if i!=9:
#         all_train_data3[i]=(all_train_data2[i] - all_train_data2[i].mean()) / all_train_data2[i].std()
#     else:
#         all_train_data3[i]=all_train_data2[i]

In [17]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,70.426493,65.832202,0.186377,0.079806,0.672361,2.283694,1.937404,7.449352,9.327724,4.251942,0.008977,15482.226219,1.051691,3.676586,12.803597,11.687312,1.046624,0.813470
std,25.536335,12.405338,0.041562,0.032614,0.176061,0.523410,0.306804,2.431622,3.291614,1.859045,0.035324,6063.422584,0.303361,0.481140,3.602422,3.020925,0.263948,0.292340
min,12.833239,0.000000,0.068067,0.000000,0.000000,0.000000,0.769032,0.000000,0.000000,0.000000,0.000000,1616.845054,0.000000,0.000000,0.096879,0.000000,0.000000,0.000000
25%,49.926550,50.952479,0.158556,0.055291,0.562804,1.938097,1.735287,5.766358,7.204513,3.000000,0.000000,10617.557930,0.852464,3.245618,9.751980,9.184518,0.857234,0.592611
50%,70.525274,63.921527,0.188618,0.076901,0.672848,2.270782,1.916995,7.472206,9.283713,4.242641,0.000000,15534.166404,1.035990,3.534372,12.099014,11.107693,1.042283,0.818152
75%,88.521007,79.522794,0.215417,0.103613,0.775804,2.651342,2.143833,9.063998,11.523345,5.477226,0.000000,20398.022735,1.243713,3.834142,15.383128,13.963600,1.233421,1.030980
max,133.380091,120.311749,0.306532,0.160805,1.192886,3.876332,2.905076,20.678088,20.721569,10.583005,0.161645,30295.187454,2.192628,7.863631,18.965279,16.819218,1.852950,1.618322


In [18]:
all_train_data3=pd.DataFrame()
for i in range(all_train_data2.shape[1]):
        all_train_data3[i]=(all_train_data2[i] - all_train_data2[i].mean()) / all_train_data2[i].std()

In [19]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.705521,0.388436,0.140427,2.136970,10.125990,12.254115,31.905469,42.709201,21.534462,0.200625,73.229167,2.769089,1.843012,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.100203,0.323505,0.104645,2.281611,6.187555,7.571422,18.703486,26.222292,16.575716,2.045443,13.361351,1.809743,0.163008,95.745881,94.697432,1.065408,1.062683
min,6.700000,0.000000,0.080000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [20]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,70.426493,65.832202,0.186377,0.079806,0.672361,2.283694,1.937404,7.449352,9.327724,4.251942,0.008977,15482.226219,1.051691,3.676586,12.803597,11.687312,1.046624,0.813470
std,25.536335,12.405338,0.041562,0.032614,0.176061,0.523410,0.306804,2.431622,3.291614,1.859045,0.035324,6063.422584,0.303361,0.481140,3.602422,3.020925,0.263948,0.292340
min,12.833239,0.000000,0.068067,0.000000,0.000000,0.000000,0.769032,0.000000,0.000000,0.000000,0.000000,1616.845054,0.000000,0.000000,0.096879,0.000000,0.000000,0.000000
25%,49.926550,50.952479,0.158556,0.055291,0.562804,1.938097,1.735287,5.766358,7.204513,3.000000,0.000000,10617.557930,0.852464,3.245618,9.751980,9.184518,0.857234,0.592611
50%,70.525274,63.921527,0.188618,0.076901,0.672848,2.270782,1.916995,7.472206,9.283713,4.242641,0.000000,15534.166404,1.035990,3.534372,12.099014,11.107693,1.042283,0.818152
75%,88.521007,79.522794,0.215417,0.103613,0.775804,2.651342,2.143833,9.063998,11.523345,5.477226,0.000000,20398.022735,1.243713,3.834142,15.383128,13.963600,1.233421,1.030980
max,133.380091,120.311749,0.306532,0.160805,1.192886,3.876332,2.905076,20.678088,20.721569,10.583005,0.161645,30295.187454,2.192628,7.863631,18.965279,16.819218,1.852950,1.618322


In [21]:
all_train_data3.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,1.019875e-13,-5.144712e-14,6.653925e-14,2.549689e-14,2.334629e-15,-6.727368e-14,-4.221841e-14,2.062856e-15,5.215490e-14,-1.573600e-14,-1.659610e-15,6.245795e-14,-9.682569e-15,-1.537578e-14,1.616595e-14,-8.086471e-15,3.354384e-14,3.181464e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.255345e+00,-5.306764e+00,-2.846587e+00,-2.447008e+00,-3.818917e+00,-4.363104e+00,-3.808196e+00,-3.063532e+00,-2.833784e+00,-2.287165e+00,-2.541260e-01,-2.286725e+00,-3.466795e+00,-7.641412e+00,-3.527271e+00,-3.868786e+00,-3.965265e+00,-2.782619e+00
25%,-8.027755e-01,-1.199461e+00,-6.693840e-01,-7.516859e-01,-6.222681e-01,-6.602794e-01,-6.587790e-01,-6.921282e-01,-6.450364e-01,-6.734331e-01,-2.541260e-01,-8.022974e-01,-6.567335e-01,-8.957229e-01,-8.471015e-01,-8.284860e-01,-7.175293e-01,-7.554874e-01
50%,3.868283e-03,-1.540203e-01,5.390355e-02,-8.905258e-02,2.767230e-03,-2.466757e-02,-6.652185e-02,9.398760e-03,-1.337080e-02,-5.003470e-03,-2.541260e-01,8.566150e-03,-5.175865e-02,-2.955772e-01,-1.955858e-01,-1.918681e-01,-1.644871e-02,1.601343e-02
75%,7.085792e-01,1.103605e+00,6.987095e-01,7.299705e-01,5.875442e-01,7.024097e-01,6.728357e-01,6.640201e-01,6.670347e-01,6.590929e-01,-2.541260e-01,8.107297e-01,6.329803e-01,3.274645e-01,7.160547e-01,7.535070e-01,7.077023e-01,7.440291e-01
max,2.465256e+00,4.391621e+00,2.890982e+00,2.483615e+00,2.956513e+00,3.042810e+00,3.154037e+00,5.440292e+00,3.461476e+00,3.405546e+00,4.321989e+00,2.443003e+00,3.760984e+00,8.702351e+00,1.710428e+00,1.698787e+00,3.054866e+00,2.753135e+00


In [22]:
from scipy import stats
import seaborn as sns

In [23]:
stats.probplot(all_train_data1[9].values,fit=stats.norm,plot=plt)
plt.show()

In [24]:
sns.distplot(np.sqrt(all_train_data1[9].values),fit=stats.norm)
plt.show()

In [25]:
stats.probplot(np.sqrt(all_train_data1[9].values),fit=stats.norm,plot=plt)
plt.show()

In [47]:
stats.probplot(all_train_data2[9].values,fit=stats.norm,plot=plt)
plt.show()

In [48]:

sns.distplot(all_train_data2[9].values,fit=stats.norm)

In [49]:
stats.probplot(all_train_data3[9].values,fit=stats.norm,plot=plt)

((array([-3.67199794, -3.43886226, -3.31047894, ...,  3.31047894,
          3.43886226,  3.67199794]),
  array([-2.28716496, -2.28716496, -2.28716496, ...,  3.3287868 ,
          3.3287868 ,  3.40554597])),
 (0.9971451001592022, -1.5947286174094872e-14, 0.9966456037784214))

In [50]:
sns.distplot(all_train_data3[9].values,fit=stats.norm)

- all_train_data1 : fix outlier 
- all_train_data2 : fix skew and kurt
- all_train_data3 : 18 feats norm

In [51]:
8897+989

9886

### just use original data to predict next pm2.5
- use all_train_data1

In [52]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data1.T.iloc[:,m*480:(m+1)*480]
    year_data3[m]=month_data3

- x shape : 12 * 479 (480-1,since next pm2.5) , 17
- y shape : t+1 pm2.5 ,12 * 479

In [53]:
year_data3[0].shape

(18, 480)

In [54]:
year_data3[m].transpose().shape

(480, 18)

In [55]:
year_data3[0].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]][0]

array([14.  ,  1.8 ,  0.51,  0.2 ,  0.9 , 16.  , 17.  , 16.  , 56.  ,
       26.  ,  0.  , 77.  ,  1.8 ,  2.  , 37.  , 35.  ,  1.4 ,  0.5 ])

In [56]:
year_data3[m].transpose()[:5,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]

array([[2.3e+01, 1.6e+00, 2.2e-01, 7.0e-02, 1.5e+00, 6.1e+00, 7.6e+00,
        1.6e+01, 3.4e+01, 2.2e+01, 0.0e+00, 7.9e+01, 1.8e+00, 1.7e+00,
        9.0e+01, 9.3e+01, 1.8e+00, 1.3e+00],
       [2.3e+01, 1.7e+00, 2.0e-01, 7.0e-02, 1.3e+00, 6.1e+00, 7.4e+00,
        1.8e+01, 1.9e+01, 1.7e+01, 0.0e+00, 7.8e+01, 1.6e+00, 1.7e+00,
        4.9e+01, 5.0e+01, 2.1e+00, 9.0e-01],
       [2.3e+01, 1.7e+00, 1.8e-01, 3.0e-02, 1.4e+00, 4.6e+00, 6.0e+00,
        2.7e+01, 2.4e+01, 1.7e+01, 0.0e+00, 8.1e+01, 2.5e+00, 1.7e+00,
        8.2e+01, 9.9e+01, 3.2e+00, 1.0e+00],
       [2.2e+01, 1.7e+00, 3.0e-01, 6.0e-02, 1.1e+00, 6.0e+00, 7.0e+00,
        3.3e+01, 2.9e+01, 2.2e+01, 0.0e+00, 8.2e+01, 3.2e+00, 1.7e+00,
        8.5e+01, 7.1e+01, 4.0e+00, 2.2e+00],
       [2.1e+01, 1.7e+00, 4.0e-01, 5.0e-02, 9.0e-01, 6.3e+00, 7.1e+00,
        2.2e+01, 3.5e+01, 2.7e+01, 0.0e+00, 7.9e+01, 1.5e+00, 1.8e+00,
        5.8e+01, 5.8e+01, 3.3e+00, 3.7e+00]])

In [57]:
year_data3[0].transpose()[1:,9][:5]

array([39., 36., 35., 31., 28.])

In [58]:
x=np.empty((12*479,18))
y=np.empty((12*479,1))

In [59]:
for m in range(12):
    x[m*479:(m+1)*479,:]=year_data3[m].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
    y[m*479:(m+1)*479,:]=year_data3[m].transpose()[1:,9:10]

In [60]:
x[:5]

array([[ 14.  ,   1.8 ,   0.51,   0.2 ,   0.9 ,  16.  ,  17.  ,  16.  ,
         56.  ,  26.  ,   0.  ,  77.  ,   1.8 ,   2.  ,  37.  ,  35.  ,
          1.4 ,   0.5 ],
       [ 14.  ,   1.8 ,   0.41,   0.15,   0.6 ,   9.2 ,   9.8 ,  30.  ,
         50.  ,  39.  ,   0.  ,  68.  ,   2.  ,   2.  ,  80.  ,  79.  ,
          1.8 ,   0.9 ],
       [ 14.  ,   1.8 ,   0.39,   0.13,   0.5 ,   8.2 ,   8.7 ,  27.  ,
         48.  ,  36.  ,   0.  ,  67.  ,   1.7 ,   2.  ,  57.  ,   2.4 ,
          1.  ,   0.6 ],
       [ 13.  ,   1.8 ,   0.37,   0.12,   1.7 ,   6.9 ,   8.6 ,  23.  ,
         35.  ,  35.  ,   0.  ,  74.  ,   1.6 ,   1.9 ,  76.  ,  55.  ,
          0.6 ,   0.3 ],
       [ 12.  ,   1.8 ,   0.35,   0.11,   1.8 ,   6.8 ,   8.5 ,  24.  ,
         25.  ,  31.  ,   0.  ,  72.  ,   1.9 ,   1.9 , 110.  ,  94.  ,
          1.7 ,   0.6 ]])

In [61]:
y[:5]

array([[39.],
       [36.],
       [35.],
       [31.],
       [28.]])

In [62]:
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [63]:
n_folds=5
for i in range(9):
    print('-----------------------------------',i,'----------------------------------------')
    lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x)
    rmse=np.sqrt(-cross_val_score(lasso,x,y,scoring='neg_mean_squared_error',cv=kf))
    score=rmse
    print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


----------------------------------- 0 ----------------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



Lasso score: 6.2479 (0.6112)

----------------------------------- 1 ----------------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



Lasso score: 6.2479 (0.6112)

----------------------------------- 2 ----------------------------------------


...

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



Lasso score: 6.2479 (0.6112)

----------------------------------- 7 ----------------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



Lasso score: 6.2479 (0.6112)

----------------------------------- 8 ----------------------------------------

Lasso score: 6.2479 (0.6112)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


#### ust sqrt of  pm2.5

In [64]:
year_data33={}
all_train_data11=all_train_data1.copy()
## ust sqrt to pm2.5
all_train_data11[9]=np.sqrt(all_train_data11[9])
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data11.T.iloc[:,m*480:(m+1)*480]
    year_data33[m]=month_data3

In [65]:
x1=np.empty((12*479,18))
y1=np.empty((12*479,1))

In [66]:
for m in range(12):
    x1[m*479:(m+1)*479,:]=year_data33[m].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
    y1[m*479:(m+1)*479,:]=year_data33[m].transpose()[1:,9:10]

In [67]:
n_folds=5
for i in range(9):
    print('-----------------------------------',i,'----------------------------------------')
    lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x1)
    rmse=np.sqrt(-cross_val_score(lasso,x1,y1,scoring='neg_mean_squared_error',cv=kf))
    score=rmse
    print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


----------------------------------- 0 ----------------------------------------

Lasso score: 0.7890 (0.0514)

----------------------------------- 1 ----------------------------------------

Lasso score: 0.7890 (0.0514)

----------------------------------- 2 ----------------------------------------

Lasso score: 0.7890 (0.0514)

----------------------------------- 3 ----------------------------------------

Lasso score: 0.7890 (0.0514)

----------------------------------- 4 ----------------------------------------

Lasso score: 0.7890 (0.0514)

----------------------------------- 5 ----------------------------------------

Lasso score: 0.7890 (0.0514)

----------------------------------- 6 ----------------------------------------

Lasso score: 0.7890 (0.0514)

----------------------------------- 7 ----------------------------------------

Lasso score: 0.7890 (0.0514)

----------------------------------- 8 ----------------------------------------

Lasso score: 0.7890 (0.0514)



### just use original data to predict next pm2.5
- use all_train_data2 & use all_train_data3

In [75]:
year_data4={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data2.T.iloc[:,m*480:(m+1)*480]
    year_data4[m]=month_data3

In [76]:
year_data5={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data3.T.iloc[:,m*480:(m+1)*480]
    year_data5[m]=month_data3

In [77]:
x2=np.empty((12*479,18))
y2=np.empty((12*479,1))

In [78]:
x3=np.empty((12*479,18))
y3=np.empty((12*479,1))

In [79]:
for m in range(12):
    x2[m*479:(m+1)*479,:]=year_data4[m].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
    y2[m*479:(m+1)*479,:]=year_data4[m].transpose()[1:,9:10]

In [80]:
for m in range(12):
    x3[m*479:(m+1)*479,:]=year_data5[m].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
    y3[m*479:(m+1)*479,:]=year_data5[m].transpose()[1:,9:10]

### use  all_train_data2

In [81]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x2)
rmse=np.sqrt(-cross_val_score(lasso,x2,y2,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


----------------------------------- start ----------------------------------------

Lasso score: 0.7845 (0.0513)



### use  all_train_data3

In [82]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x3)
rmse=np.sqrt(-cross_val_score(lasso,x3,y3,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- start ----------------------------------------

Lasso score: 0.4219 (0.0278)



### x use all_train_data3 and y use all_train_data1

In [66]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x3)
rmse=np.sqrt(-cross_val_score(lasso,x3,y,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- start ----------------------------------------

Lasso score: 7.1592 (0.7185)



In [67]:
lasso.fit(x3,y3)
y3_hat=lasso.predict(x3).reshape(-1,1)

In [68]:
lasso.fit(x2,y2)
y2_hat=lasso.predict(x2).reshape(-1,1)

In [69]:
lasso.fit(x,y)
y_hat=lasso.predict(x2).reshape(-1,1)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [70]:
x4=np.empty((12*479,3))
x4[:,0:1]=y_hat
x4[:,1:2]=y2_hat
x4[:,2:3]=y3_hat

In [71]:
x4[:5]

array([[-407.12615151,    5.05920993,    0.42210318],
       [-300.55996841,    5.93881557,    0.93289554],
       [-290.18052625,    5.65313117,    0.76514916],
       [-369.36139029,    5.55579873,    0.7104485 ],
       [-346.09756532,    5.24326854,    0.52891552]])

In [72]:
y3[:5]

array([[1.11326216],
       [0.97051618],
       [0.92162079],
       [0.71867781],
       [0.55771628]])

In [73]:
y2[:5]

array([[6.244998  ],
       [6.        ],
       [5.91607978],
       [5.56776436],
       [5.29150262]])

In [74]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x4)
rmse=np.sqrt(-cross_val_score(lasso,x4,y3,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- start ----------------------------------------

Lasso score: 0.4022 (0.0212)



In [75]:
all_train_data1.corr()[9]

0    -0.027470
1     0.330326
2     0.284273
3     0.288906
4     0.029632
5     0.451861
6     0.377964
7     0.357789
8     0.772313
9     1.000000
10   -0.062040
11   -0.259209
12    0.368880
13    0.394145
14    0.185643
15    0.157551
16   -0.083962
17   -0.049691
Name: 9, dtype: float64

In [76]:
all_train_data2.corr()[9]

0     0.006680
1     0.301785
2     0.469347
3     0.346056
4     0.012029
5     0.450559
6     0.398957
7     0.331932
8     0.719931
9     1.000000
10   -0.112281
11   -0.281043
12    0.360808
13    0.373601
14    0.182083
15    0.149513
16   -0.073194
17   -0.026756
Name: 9, dtype: float64

In [77]:
all_train_data3.corr()[9]

0     0.006680
1     0.301785
2     0.469347
3     0.346056
4     0.012029
5     0.450559
6     0.398957
7     0.331932
8     0.719931
9     1.000000
10   -0.112281
11   -0.281043
12    0.360808
13    0.373601
14    0.182083
15    0.149513
16   -0.073194
17   -0.026756
Name: 9, dtype: float64